In [14]:
import pydicom
import os

dir = '/Users/oovamoyo/Downloads/МРТ Липатова ТА - БП+ЭТ/2023'
fname = os.path.join(dir, 'DICOMDIR')
dcom = pydicom.dcmread(fname)

In [15]:
dcom.DirectoryRecordSequence

<Sequence, length 416>

In [19]:
print(len(dcom.DirectoryRecordSequence))
# 1. create a new set
record_types = set()

# 2. populate with DirectoryRecordType
for rec in dcom.DirectoryRecordSequence:
    record_types.add(rec.DirectoryRecordType)

# Print the unique DirectoryRecordTypes
print(record_types)


for record in dcom.DirectoryRecordSequence:
    if record.DirectoryRecordType == 'PATIENT':
        print(record)
        # path = os.path.join(*record.ReferencedFileID)
        # print(path)

416
{'PATIENT', 'IMAGE', 'STUDY', 'SERIES'}
(0004,1400) Offset of the Next Directory Record UL: 0
(0004,1410) Record In-use Flag                  US: 65535
(0004,1420) Offset of Referenced Lower-Level Di UL: 530
(0004,1430) Directory Record Type               CS: 'PATIENT'
(0008,0005) Specific Character Set              CS: 'ISO_IR 100'
(0010,0010) Patient's Name                      PN: 'Lipatova T.A.'
(0010,0020) Patient ID                          LO: 'l'
(0010,0030) Patient's Birth Date                DA: '19550310'
(0010,0040) Patient's Sex                       CS: 'F'


In [ ]:
import yadisk

YA_TOKEN=os.getenv('YA_TOKEN')
client = yadisk.Client(token=YA_TOKEN)

print('Token is valid?', client.check_token())

Token is valid? True


In [31]:
client.mkdir('Загрузки/MRT_PNGs')

<SyncResourceLinkObject{
    'href': 'https://cloud-api.yandex.net/v1/disk/resources?path=disk%3A%2F%D0%97%D0%B0%D0%B3%D1%80%D1%83%D0%B7%D0%BA%D0%B8%2FMRT_PNGs',
    'method': 'GET',
    'templated': False,
    'path': 'disk:/Загрузки/MRT_PNGs'})>

In [ ]:
client.upload(os.path.join(os.getcwd(), 'artifacts/MPT_AAA_PNGs/2024'), 'Загрузки/MRT_PNGs/2024', overwrite=True)

IsADirectoryError: [Errno 21] Is a directory: '/Users/oovamoyo/PycharmProjects/brain_pd_et/artifacts/MPT_AAA_PNGs/2024'

In [37]:
from PIL import Image
import numpy as np

im = Image.open('artifacts/MPT_AAA_PNGs/2024/DICOM/PA000000/ST000000/SE000004/IM000001.png')
arr = np.asarray(im)
arr.shape

(128, 128)